# AI Model for English Accent Classification on Audio Streams Extracted from Videos 

**N8N AI Workflow Automation Dashboard Self-hosted:** [https://ai.loquisoft.com](https://ai.loquisoft.com)

---

This notebook demonstrates a working proof-of-concept for classifying English accents from spoken audio in video files.  
It uses state-of-the-art AI (OpenAI Whisper and GPT-4) for transcription and accent reasoning.

## Instructions
1. Enter your OpenAI API key.
2. Enter or use the sample public video URL (MP4 or Loom, direct links work best).
3. The notebook will:
    - Download the video
    - Extract audio
    - Transcribe the audio (Whisper)
    - Analyze and classify the accent, returning:
        - Accent type (e.g., British, American, Australian, etc.)
        - Confidence score (0–100%)
        - Short explanation

## Requirements
- OpenAI API access (key required)
- Video must be public and downloadable (direct link or Loom)

## Cost Transparency

- **Whisper-1 transcription:** Approximately \$0.006 per audio minute ([OpenAI Pricing](https://openai.com/pricing))
- **GPT-4 prompt/response:** Depends on transcript length; short interview clips usually cost a few cents per run.
- **For this demo:** Using the sample video will incur negligible cost (less than \$0.02 total).

## Deliverables
- Fully functional and clear script
- Logical, modern AI approach
- Clear outputs and setup

---

## To run:
1. Run each cell in order.
2. Outputs will appear below each step.


In [ ]:
#@title Install required libraries (quietly)
try:
    import moviepy
except ImportError:
    !apt-get update -qq && apt-get install -y -qq ffmpeg
    !pip install --upgrade yt-dlp moviepy openai --quiet
    !pip install -q openai-whisper pytube ffmpeg-python torch

In [ ]:
import openai
print("OpenAI version:", openai.__version__)

OpenAI version: 1.78.1


## 1. Enter OpenAI API Key and Video URL

In [ ]:
#@title Enter Your OpenAI API Key and Video URL
#
# For production: use getpass and input for security
# import getpass
# openai_api_key = getpass.getpass("Enter your OpenAI API key: ")
# video_url = input("Enter a public video URL (direct MP4 or Loom): ")

# For demonstration/testing, values are hardcoded below
openai_api_key = "Enter_your_API_Key_here"  # <-- Replace with your actual OpenAI API key
video_url = "https://loquisoft-media.s3.amazonaws.com/file/sampleMonologue1.mp4"

print(f"Using video URL: {video_url}")

Using video URL: https://loquisoft-media.s3.amazonaws.com/file/sampleMonologue1.mp4


## 2. Download Video (with Exception Handling)

In [ ]:
#@title Download Video
import yt_dlp
import os

download_error = None
video_file = None

try:
    ydl_opts = {"outtmpl": "input_video.%(ext)s"}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])
    video_file = [f for f in os.listdir() if f.startswith("input_video")][0]
    print(f"Downloaded video: {video_file}")
except Exception as e:
    download_error = str(e)
    print(f"Error downloading video: {download_error}")

[generic] Extracting URL: https://loquisoft-media.s3.amazonaws.com/file/sampleMonologue1.mp4
[generic] sampleMonologue1: Downloading webpage
[info] sampleMonologue1: Downloading 1 format(s): mp4
[download] input_video.mp4 has already been downloaded
[download] 100% of    6.11MiB
Downloaded video: input_video.mp4


## 3. Extract Audio (with Exception Handling)

In [ ]:
#@title Extract Audio from Video
from moviepy.editor import VideoFileClip

audio_file = None
try:
    clip = VideoFileClip(video_file)
    audio_file = "extracted_audio.wav"
    clip.audio.write_audiofile(audio_file, verbose=False, logger=None)
    print(f"Extracted audio saved to {audio_file}")
except Exception as e:
    print(f"Error extracting audio: {e}")

## 4. Transcribe Audio Using Whisper

In [ ]:
import whisper

transcription = None
try:
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    transcription = result["text"]
    print("Transcription completed.")
    print(f"Transcript sample (first 300 chars): {transcription[:300]}...")
except Exception as e:
    print(f"Error during transcription: {e}")

## 5. Classify Accent Using GPT-4

In [ ]:
openai.api_key = openai_api_key

prompt = f"""
You are a linguistic AI assistant tasked with identifying the speaker's English accent based on the following transcript:

{transcription}

Please classify the accent into one of the following categories:
- British
- American
- Australian
- Canadian
- Indian
- Irish
- Scottish
- Welsh
- Other (please specify)

Provide your answer as a JSON object with fields:
  "accent": <accent name>,
  "confidence": <percentage number 0-100>,
  "explanation": <short reasoning>
"""

try:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    classification = response.choices[0].message.content
    print("Accent classification response:")
    print(classification)
except Exception as e:
    print(f"Error calling OpenAI GPT-4 API: {e}")